# Foxtrot-Core · Interactive *Fabric Mapping* Tutorial

This mini-guide walks you through the complete box-tiling and
coordinate-labelling workflow shipped with **foxtrot-core** – tested on
the *XC7A100T-CSG324* FPGA family.

> **Setup (do this first)**
> **Required:** install **foxtrot-core with the _analysis_ extra**  
> `pip install -U "foxtrot-core[analysis]"`
>
> This brings in the stack used by `fabmap.*` (NumPy, pandas, ipywidgets, ipympl, matplotlib, etc.) so widgets and interactive plots work out of the box.  
> *Tip:* enable interactive figures with `%matplotlib widget` (provided by `ipympl`).

The practical steps:

1. **Load an offset mask** (the raw bit-offsets you want to explore).  
2. **Define a template + area** and run the vectorised *template-tiler*.  
3. **Assign human-friendly coordinates** to each orange box.  
4. **Merge several labelled JSON files** and visually inspect the result.  

We provide several real-world data sets so you can start experimenting
immediately.

## 1 · Example data files

All paths are relative to your project root.

| Purpose | Path | Notes |
|---------|------|-------|
| **Full LUT mask** – every LUT configuration bit for the XC7A100T part | `projects/fabmap/0-template/offsets/XC7A100TCSG324_FULL_LUTMASK_AddedMissingLUT.off` | Use **frame size 3232**. |
| Eight-LUT demo | `projects/fabmap/0-template/offsets/XC7A100TCSG324_EIGHT_LUTMASK.off` | Matches placement CSV below. |
| Single-LUT demo | `projects/fabmap/0-template/offsets/XC7A100TCSG324_SINGLE_LUTMASK.off` | Matches placement CSV below. |
| Placement coordinates (Eight LUTs) | `projects/fabmap/0-template/placements/XC7A100TCSG324_EIGHT_LUTMASK.csv` | Confirms map orientation. |
| Placement coordinates (Single LUT) | `projects/fabmap/0-template/placements/XC7A100TCSG324_SINGLE_LUTMASK.csv` | idem. |
| **Pre-segmented** (un-labelled) LUT boxes | `projects/fabmap/0-template/segments/unlabeled/luts/XC7A100TCSG324.json` | Every LUT boxed – great for orientation. |
| **Pre-segmented** (un-labelled) Slice boxes | `projects/fabmap/0-template/segments/unlabeled/slices/XC7A100TCSG324.json` | One orange box per SLICE. |
| 8 × split labelled Slice files | `projects/fabmap/0-template/segments/labeled/slices/XC7A100TCSG324_SPLIT_<N>.json` | Coordinates applied per region. |
| **Final merged** Slice labels | `projects/fabmap/0-template/segments/labeled/slices/XC7A100TCSG324.json` | What we’ll get after *merge_runner*. |


In [ ]:
# │ codecell 1 │
%matplotlib widget

# Fallback (if ipympl is unavailable)
# %matplotlib tk

from foxtrot_core import __version__ as fc_version
print(f"Foxtrot-Core {fc_version} loaded ✓")


## 2 Loading the Offset File

During the **BitGen** fuzz-campaign we toggled every LUT on a bare
*XC7A100T-CSG324* design and logged the set bits. Vivado omitted **one
LUT** due to DRC limitations, so we manually inspected the 64-bit gap
in the frame stream, added the missing offsets, and saved the result to  
`XC7A100TCSG324_FULL_LUTMASK_AddedMissingLUT.off`.

You will also find:

* **Eight-LUT** and **Single-LUT** masks – handy orientation sanity-checks
  because their *fabric* coordinates were logged during BitGen and live
  in the matching `placements/*.csv` files.


In [ ]:
# │ codecell 2 │
from foxtrot_core.fabmap.ui.sources import offset_loader

ui_loader, get_df, fs_w = offset_loader()   # ⇦ widget + helpers

## 3 Running the Tiler

Once you see the green **✓ loaded** message the scatter‑plot widget
activates automatically.

### Choose your test data

| Mask file | Size | Why it’s useful |
|-----------|------|-----------------|
| `…_FULL_LUTMASK_AddedMissingLUT.off` | **63400 LUTs** | Stress‑test anchor detection – every LUT bit is present. |
| `…_EIGHT_LUTMASK.off` | **8 LUTs** | Check **fabric orientation** – use together with the placement CSV to verify X/Y mapping. |
| `…_SINGLE_LUTMASK.off` | **1 LUT** | Ultra‑quick sanity test; expect exactly one orange box after tiling. |

*(All live under `projects/fabmap/0-template/offsets/` – frame‑size is
**3232** for this device.)*

---

### Five‑step workflow

1. **Template →** select the *smallest* red pattern that still captures
   the feature of interest (e.g. \
   • **64 bits** → one LUT or \
   • **256 bits** → one SLICE ).

2. **Area →** drag a green rectangle around the fabric region you wish
   to tile – the larger the better coverage.

3. Click **Tile Grid** → orange boxes appear, one per template match.

4. Use **Shift / Pad** to refine box placement:  
   * shift moves the grid; pad shrinks/expands each box.

5. *(Optional)* **Export Boxes** – saves to
   `boxes_<timestamp>.json` for later import.

---

### Extra experiments

* **Orientation check**:  
  Load `EIGHT_LUTMASK`. After tiling, each orange box should overlay the eight blue dots from the placement file exactly – confirming X/Y axes.

* **Import pre‑segmented boxes**:  
  Click **Load Boxes** and choose  
  `segments/unlabeled/luts/XC7A100TCSG324.json` or  
  `segments/unlabeled/slices/XC7A100TCSG324.json`  
  to visualise the *auto‑generated* full‑device slice masks.


In [ ]:
# │ codecell 3 │
from foxtrot_core.fabmap.ui.runners.tiler_runner import tiler_runner

runner, get_boxes = tiler_runner(get_df, fs_w)


## 4 Running the Coordinate Mapper

1. Click **Transfer boxes** once tiling completes (blue progress bar).  
2. Use pattern drop‑downs – or free‑type – to craft row/col labels.  
3. Prefix + Separator let you build things like `SLICE_X123Y456`.  
4. **Apply labels** shows a live preview table.  
5. **Export JSON** when happy (repeat for other regions if needed).


In [ ]:
# │ codecell 4 │
from foxtrot_core.fabmap.ui.runners.coords_runner import coords_runner

coords_ui, get_labels = coords_runner(get_boxes, fs_w)


# 5 Running the JSON Merger

Drag all eight `XC7A100TCSG324_SPLIT_<N>.json` files,  
press **Merge & Visualise**.

* Duplicate labels, overlapping offsets or unlabeled gaps are flagged.  
* Hover any blue point to see its coordinate.  
* Finish with **Export merged JSON** – see
  `XC7A100TCSG324.json` for the golden reference.


In [ ]:
# │ codecell 5 │
from foxtrot_core.fabmap.ui.runners import merge_runner

merge_ui, get_merged = merge_runner(frame_size_widget=fs_w)
